In [1]:
import os

In [4]:
os.getcwd()

'c:\\Users\\Cynthia\\projects\\Hospital_readmission'

In [3]:
os.chdir("../")

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from src.constants import *
from src.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:

    def __init__(self, 
                 config_filepath=CONFIG_FILE_PATH, 
                 params_filepath=PARAMS_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)

        create_directories([self.config.artifacts_roots])

    def get_data_ingestion_configuration(self) -> DataIngestionConfig:
        config=self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
                                            root_dir=config.root_dir,
                                            source_url= config.source_url,
                                            local_data_file=config.local_data_file,
                                            unzip_dir=config.unzip_dir
                                         )
        return data_ingestion_config

In [12]:
import os
import urllib.request as request
from zipfile import ZipFile
from src.logger import logging
from src.utils.common import get_size

class DataIngestion:
    def __init__(self, config:DataIngestionConfig):
        self.config=config

    def download_file(self):
        if (not os.path.exists(self.config.local_data_file)):
            filename, headers = request.urlretrieve(
                url=self.config.source_url,
                filename=self.config.local_data_file
            )
            logging.info(f"{filename} download! with the following info \n{headers}")
        else:
            logging.info(f"File already exist of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zipfile(self):

        unzip_path=self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with ZipFile(self.config.local_data_file, "r") as zip_file:
            zip_file.extractall(unzip_path)

In [13]:
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_configuration()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zipfile()
except Exception as e:
    raise e

{'artifacts_roots': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_url': 'https://github.com/okanina/Hospital_readmission/raw/main/diabetic_data.zip', 'local_data_file': 'artifacts/data_ingestion/diabetic_data.zip', 'unzip_dir': 'artifacts/data_ingestion'}}
{'df_ingestion': {'root_dir': 'artifacts/data_ingestion', 'unzip_dir': 'artifacts/data_ingestion'}}
